# Neural Networks and Keras
## Nate Damen - created 09/28/2020 - updated xxxxxxx

In [9]:
import time
import os
import tensorflow as tf
import numpy as np
from sklearn.metrics import mean_squared_error

In [10]:
root_logdir = os.path.join(os.curdir, "logs")


def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)


run_logdir = get_run_logdir()
run_logdir

'./logs/run_2020_10_03-01_01_55'

In [11]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10)
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    "mnist_model.h5", monitor='val_loss', save_best_only=True)
tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir)

Going to be making a neural network to classify the mnist data set

In [12]:
(x_train_full, y_train_full), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 1s 0us/step


In [13]:
x_valid, x_train = x_train_full[:5000] / 255., x_train_full[5000:] / 255.
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
x_test = x_test / 255.
# dividing by 255 scales the data to 0-1

In [14]:
x_train.shape

(55000, 28, 28)

Below is the built model

In [7]:
input_layer1 = tf.keras.layers.Input(shape=x_train.shape[1:])

flatten = tf.keras.layers.Flatten()(input_layer1)

hidden1 = tf.keras.layers.Dense(392, activation="relu")(flatten)
hidden2 = tf.keras.layers.Dense(196, activation="relu")(hidden1)
#hidden3 = tf.keras.layers.Dense(784, activation="relu")(hidden2)

output = tf.keras.layers.Dense(10, activation="softmax")(hidden2)

model = tf.keras.Model(inputs=[input_layer1], outputs=[output])

In [8]:
opt = tf.keras.optimizers.SGD(learning_rate=2e-1)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=opt, metrics=["accuracy"])

In [9]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 392)               307720    
_________________________________________________________________
dense_1 (Dense)              (None, 196)               77028     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1970      
Total params: 386,718
Trainable params: 386,718
Non-trainable params: 0
_________________________________________________________________


In [10]:
run_logdir = get_run_logdir()

early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10)
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    "mnist_model.h5", monitor='val_loss', save_best_only=True)
tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir)

history = model.fit(x_train, y_train, batch_size=32, epochs=200, validation_data=(
    x_valid, y_valid), callbacks=[early_stopping_cb, checkpoint_cb, tensorboard_cb])

Epoch 1/200
   1/1719 [..............................] - ETA: 0s - loss: 2.3749 - accuracy: 0.1250WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2299 - accuracy: 0.9298 - val_loss: 0.1228 - val_accuracy: 0.9644
Epoch 2/200
1719/1719 [==============================] - 5s 3ms/step - loss: 0.0899 - accuracy: 0.9719 - val_loss: 0.0791 - val_accuracy: 0.9758
Epoch 3/200
1719/1719 [==============================] - 5s 3ms/step - loss: 0.0575 - accuracy: 0.9815 - val_loss: 0.0659 - val_accuracy: 0.9802
Epoch 4/200
1719/1719 [==============================] - 5s 3ms/step - loss: 0.0416 - accuracy: 0.9862 - val_loss: 0.0744 - val_accuracy: 0.9788
Epoch 5/200
1719/1719 [==============================] -

In [11]:
y_preds = model.predict(x_test)
y_labeled = list(map(lambda x: np.argmax(x), y_preds))

In [12]:
mse = mean_squared_error(y_test, y_labeled)
mse

0.2749

In [13]:
confusion = tf.math.confusion_matrix(
    labels=tf.constant(y_test),
    predictions=tf.constant(y_labeled),
    num_classes=10)

print(confusion)

tf.Tensor(
[[ 973    1    0    2    0    0    1    0    1    2]
 [   1 1128    0    1    0    0    2    1    2    0]
 [   4    1 1014    3    1    0    1    3    5    0]
 [   0    0    4  996    0    3    0    3    3    1]
 [   1    0    3    0  964    0    3    2    0    9]
 [   2    0    0    9    0  873    2    2    3    1]
 [   2    2    1    1    4    3  944    0    1    0]
 [   2    1    7    0    0    0    0 1009    4    5]
 [   1    0    2    3    2    1    1    2  960    2]
 [   2    2    0    2    8    2    1    5    1  986]], shape=(10, 10), dtype=int32)


In [14]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 0s 2ms/step - loss: 0.0732 - accuracy: 0.9847


[0.0731855258345604, 0.9847000241279602]

In [15]:
model2 = tf.keras.models.load_model("mnist_model.h5")  # rollback to best model
model2.load_weights("mnist_model.h5")
model2.evaluate(x_test, y_test)

313/313 [==============================] - 0s 2ms/step - loss: 0.0666 - accuracy: 0.0991


[0.06656353920698166, 0.09910000115633011]

In [16]:
y_preds = model2.predict(x_test)
y_labeled = list(map(lambda x: np.argmax(x), y_preds))
mse = mean_squared_error(y_test, y_labeled)
mse

0.359

In [17]:
confusion = tf.math.confusion_matrix(
    labels=tf.constant(y_test),
    predictions=tf.constant(y_labeled),
    num_classes=10)

print(confusion)

tf.Tensor(
[[ 970    1    0    0    0    0    5    0    1    3]
 [   0 1128    0    1    0    0    2    1    3    0]
 [   3    1 1012    3    2    0    0    4    7    0]
 [   1    0    4  991    0    5    0    4    4    1]
 [   2    0    3    0  967    0    3    3    0    4]
 [   2    0    0    8    0  872    3    1    5    1]
 [   5    3    2    1    4    2  940    0    1    0]
 [   2    2    5    2    0    0    0 1004    5    8]
 [   4    0    1    1    3    3    2    2  956    2]
 [   2    2    0    4   13    2    2    6    4  974]], shape=(10, 10), dtype=int32)


In [ ]:
%load_ext tensorboard
%tensorboard --logdir=./logs --port=8008
#!load_ext tensorboard
#!tensorboard --logdir=./logs --port=8008